In [1]:
if 0 :
    %matplotlib inline
else :
    %matplotlib notebook

#  Import  libraries

In [2]:
import sys
import os
module_path = os.path.abspath('.') +"\\_scripts"
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)
from _00_Import_packages_git3 import *

C:\Users\ng947ac\Documents\Daniel\jupyter\SoSTrades\jupyter_doc\_scripts
C:\Users\ng947ac\Documents\Daniel\git\workspace
C:\Users\ng947ac\Documents\Daniel\git\workspace\sosgemseo\src\
C:\Users\ng947ac\Documents\Daniel\git\workspace\sostrades-core


In [3]:
from numpy import array
import pandas as pd
from sos_trades_core.execution_engine.execution_engine import ExecutionEngine
from numpy.testing import assert_array_equal, assert_array_almost_equal  # @UnresolvedImport
import os
from gemseo.core.mdo_scenario import MDOScenario

# SoSDoeScenario setUp

In [4]:
study_name = 'doe'
ns = f'{study_name}'
sc_name = "SellarDoeScenario"
c_name = "SellarCoupling"
print(sc_name)

SellarDoeScenario


See §2 for Discipmline description

In [5]:
dspace_dict = {'variable': ['x', 'z', 'y_1', 'y_2'],
               'value': [[1.], [5., 2.], [1.], [1.]],
               'lower_bnd': [[0.], [-10., 0.], [-100.], [-100.]],
               'upper_bnd': [[10.], [10., 10.], [100.], [100.]],
               'enable_variable': [True, True, True, True],
               'activated_elem': [[True], [True, True], [True], [True]]}

In [6]:
dspace = pd.DataFrame(dspace_dict)
dspace

,variable,value,lower_bnd,upper_bnd,enable_variable,activated_elem
0,x,[1.0],[0.0],[10.0],True,[True]
1,z,"[5.0, 2.0]","[-10.0, 0.0]","[10.0, 10.0]",True,"[True, True]"
2,y_1,[1.0],[-100.0],[100.0],True,[True]
3,y_2,[1.0],[-100.0],[100.0],True,[True]


In [7]:
from tempfile import gettempdir
root_dir = gettempdir()
print(root_dir)
dspace.to_csv(root_dir+'\\ds.csv')

C:\Temp


In [8]:
repo = 'sos_trades_core.sos_processes.test'
proc_name = 'test_sellar_doe'

# Doe scenario execution

# 1 Set Doe optim scenario Custom DoE from file

In [9]:
#my_formulation = "DisciplinaryOpt"
my_formulation = "MDF"
#my_formulation = "IDF"
#my_formulation =  "BiLevel "

In [10]:
from gemseo.api import (
    configure_logger,
    get_algorithm_options_schema,
    get_available_doe_algorithms,
)
get_available_doe_algorithms()

['CustomDOE',
 'DiagonalDOE',
 'OT_SOBOL',
 'OT_RANDOM',
 'OT_HASELGROVE',
 'OT_REVERSE_HALTON',
 'OT_HALTON',
 'OT_FAURE',
 'OT_MONTE_CARLO',
 'OT_FACTORIAL',
 'OT_COMPOSITE',
 'OT_AXIAL',
 'OT_OPT_LHS',
 'OT_LHS',
 'OT_LHSC',
 'OT_FULLFACT',
 'OT_SOBOL_INDICES',
 'fullfact',
 'ff2n',
 'pbdesign',
 'bbdesign',
 'ccdesign',
 'lhs']

In [11]:
my_doe_algo = "CustomDOE"

In [12]:
print("\n Sellar doe solution check with ",my_formulation ," formulation")


 Sellar doe solution check with  MDF  formulation


In [13]:
exec_eng = ExecutionEngine(study_name)
factory = exec_eng.factory
builder = factory.get_builder_from_process(repo=repo,mod_id=proc_name)
exec_eng.factory.set_builders_to_coupling_builder(builder)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


In [14]:
exec_eng.display_treeview_nodes()

INFO:SoS.EE:Nodes representation for Treeview doe
|_ doe
	|_ SellarDoeScenario
		|_ Sellar_Problem
		|_ Sellar_2
		|_ Sellar_1


'Nodes representation for Treeview doe\n|_ doe\n\t|_ SellarDoeScenario\n\t\t|_ Sellar_Problem\n\t\t|_ Sellar_2\n\t\t|_ Sellar_1'

In [15]:
exec_eng.dm.get_data('doe.sub_mda_class')['value']

'MDAJacobi'

File '.\\' + 'X_pd.csv' was created from ex_03.1_doe_scenario_lhs_fullfact.ipynb

In [16]:
X_pd = pd.read_csv('.\\' + 'X_pd.csv')

In [17]:
X_pd

,doe.SellarDoeScenario.x,doe.SellarDoeScenario.z,doe.SellarDoeScenario.z.1
0,0.000000,0.000000,1.000000
1,9.379764,1.515949,5.631736
2,8.886448,-1.196271,6.523436
3,3.713714,-1.994758,4.822571
4,0.041702,1.749067,3.617234
...,...,...,...
96,6.661678,5.719822,7.645988
97,2.358656,-6.389327,5.297002
98,3.011475,8.776000,9.837792
99,4.804455,-2.413481,1.131552


Possible algo options:

In [18]:
from gemseo.algos.doe.doe_factory import DOEFactory
driver_lib = DOEFactory().create(my_doe_algo)
driver_lib.init_options_grammar(my_doe_algo)
algo_options_keys = list(driver_lib.opt_grammar.properties_dict.keys())
print(algo_options_keys)

['doe_file', 'samples', 'delimiter', 'comments', 'skiprows', 'eval_jac', 'n_processes', 'max_time', 'wait_time_between_samples']


In [19]:
#-- set up disciplines in Scenario
disc_dict = {}
# Doe inputs
disc_dict[f'{ns}.SellarDoeScenario.n_samples'] = 100
# 'lhs', 'CustomDOE', 'fullfact', ...
disc_dict[f'{ns}.SellarDoeScenario.algo'] = my_doe_algo
disc_dict[f'{ns}.SellarDoeScenario.design_space'] = dspace

disc_dict[f'{ns}.SellarDoeScenario.formulation'] = my_formulation
disc_dict[f'{ns}.SellarDoeScenario.objective_name'] = 'obj'
#disc_dict[f'{ns}.SellarDoeScenario.ineq_constraints'] = [f'c_1', f'c_2']
disc_dict[f'{ns}.SellarDoeScenario.algo_options'] = {
                                #'samples': X_pd,
                                #'samples': X_pd.to_numpy(),
                                'skiprows': 2,
                                'doe_file': '.\\' + 'X_pd.csv'
                                                    }
disc_dict

{'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x       [1.0]         [0.0]        [10.0]             True   
 1        z  [5.0, 2.0]  [-10.0, 0.0]  [10.0, 10.0]             True   
 2      y_1       [1.0]      [-100.0]       [100.0]             True   
 3      y_2       [1.0]      [-100.0]       [100.0]             True   
 
   activated_elem  
 0         [True]  
 1   [True, True]  
 2         [True]  
 3         [True]  ,
 'doe.SellarDoeScenario.formulation': 'MDF',
 'doe.SellarDoeScenario.objective_name': 'obj',
 'doe.SellarDoeScenario.algo_options': {'skiprows': 2,
  'doe_file': '.\\X_pd.csv'}}

Remark: in fact it is a <b> DoE optim </b> (with 'obj' variable taken as obj function) ! Here dspace variable was used.

In [20]:
exec_eng.get_anonimated_data_dict()

{'<study_ph>.sub_mda_class': {'type': 'string',
  'possible_values': ['MDAJacobi',
   'MDAGaussSeidel',
   'MDANewtonRaphson',
   'PureNewtonRaphson',
   'MDAQuasiNewton',
   'GSNewtonMDA',
   'GSPureNewtonMDA',
   'GSorNewtonMDA',
   'MDASequential',
   'GSPureNewtonorGSMDA'],
  'default': 'MDAJacobi',
  'numerical': True,
  'io_type': 'in',
  'type_metadata': None,
  'var_name': 'sub_mda_class',
  'user_level': 1,
  'range': None,
  'unit': None,
  'description': None,
  'discipline_full_path_list': [],
  'visibility': 'Local',
  'value': 'MDAJacobi',
  'coupling': False,
  'optional': False,
  'meta_input': False,
  'editable': True,
  'ns_reference': <sos_trades_core.execution_engine.namespace.Namespace at 0x1e73a780160>,
  'model_origin': 'cd4a740c-012d-4015-a1bc-f07c029b0014',
  'disciplines_dependencies': ['cd4a740c-012d-4015-a1bc-f07c029b0014']},
 '<study_ph>.max_mda_iter': {'type': 'int',
  'default': 30,
  'numerical': True,
  'io_type': 'in',
  'type_metadata': None,
  'var_

In [21]:
disc_dict

{'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x       [1.0]         [0.0]        [10.0]             True   
 1        z  [5.0, 2.0]  [-10.0, 0.0]  [10.0, 10.0]             True   
 2      y_1       [1.0]      [-100.0]       [100.0]             True   
 3      y_2       [1.0]      [-100.0]       [100.0]             True   
 
   activated_elem  
 0         [True]  
 1   [True, True]  
 2         [True]  
 3         [True]  ,
 'doe.SellarDoeScenario.formulation': 'MDF',
 'doe.SellarDoeScenario.objective_name': 'obj',
 'doe.SellarDoeScenario.algo_options': {'skiprows': 2,
  'doe_file': '.\\X_pd.csv'}}

In [22]:
#exec_eng.dm.set_values_from_dict(disc_dict)
exec_eng.load_study_from_input_dict(disc_dict)

INFO:SoS.EE:configuring ...
INFO:SoS.EE:configuring ...
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
ERROR:SoS.EE.DataManager:Variable: doe.SellarDoeScenario.x value is not set!
ERROR:SoS.EE.DataManager:Variable: doe.SellarDoeScenario.z value is not set!
ERROR:SoS.EE.DataManager:Variable: doe.SellarDoeScenario.Sellar_Problem.local_dv value is not set!


In [23]:
exec_eng.dm.get_data_dict_values()

{'doe.sub_mda_class': 'MDAJacobi',
 'doe.max_mda_iter': 30,
 'doe.n_processes': 1,
 'doe.chain_linearize': False,
 'doe.tolerance': 1e-06,
 'doe.use_lu_fact': False,
 'doe.warm_start': False,
 'doe.acceleration': 'm2d',
 'doe.warm_start_threshold': -1,
 'doe.n_subcouplings_parallel': 1,
 'doe.tolerance_gs': 10.0,
 'doe.relax_factor': 0.99,
 'doe.epsilon0': 1e-06,
 'doe.linear_solver_MDO': 'GMRES',
 'doe.linear_solver_MDO_preconditioner': 'None',
 'doe.linear_solver_MDO_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.linear_solver_MDA': 'GMRES',
 'doe.linear_solver_MDA_preconditioner': 'None',
 'doe.linear_solver_MDA_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.group_mda_disciplines': False,
 'doe.authorize_self_coupled_disciplines': False,
 'doe.linearization_mode': 'auto',
 'doe.cache_type': 'SimpleCache',
 'doe.cache_file_path': None,
 'doe.debug_mode': '',
 'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space

# 2 Display Disciplines I/O variables

In [24]:
sellar1 = exec_eng.dm.get_disciplines_with_name("doe." + "SellarDoeScenario.Sellar_1")[0]

In [25]:
type(sellar1)

sos_trades_core.sos_wrapping.test_discs.sellar.Sellar1

In [26]:
DESC_IN = sellar1.DESC_IN
DESC_OUT = sellar1.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [27]:
DESC_IN_df

,type,visibility,namespace
x,float,Shared,ns_OptimSellar
y_2,float,Shared,ns_OptimSellar
z,array,Shared,ns_OptimSellar


In [28]:
DESC_OUT_df

,type,visibility,namespace
y_1,float,Shared,ns_OptimSellar
y_1_bis,float,Shared,ns_OptimSellar


In [29]:
sellar2 = exec_eng.dm.get_disciplines_with_name("doe." + "SellarDoeScenario.Sellar_2")[0]

In [30]:
DESC_IN = sellar2.DESC_IN
DESC_OUT = sellar2.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [31]:
DESC_IN_df

,type,visibility,namespace,default
y_1,float,Shared,ns_OptimSellar,NaN
z,array,Shared,ns_OptimSellar,NaN
debug_mode_sellar,bool,Shared,ns_OptimSellar,False


In [32]:
DESC_OUT_df

,type,visibility,namespace
y_2,float,Shared,ns_OptimSellar
y_2_bis,float,Shared,ns_OptimSellar


In [33]:
sellarP = exec_eng.dm.get_disciplines_with_name("doe." + "SellarDoeScenario.Sellar_Problem")[0]

In [34]:
DESC_IN = sellarP.DESC_IN
DESC_OUT = sellarP.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [35]:
DESC_IN_df

,type,visibility,namespace
x,float,Shared,ns_OptimSellar
y_1,float,Shared,ns_OptimSellar
y_2,float,Shared,ns_OptimSellar
z,array,Shared,ns_OptimSellar
local_dv,float,NaN,NaN


In [36]:
DESC_OUT_df

,type,visibility,namespace
c_1,array,Shared,ns_OptimSellar
c_2,array,Shared,ns_OptimSellar
obj,array,Shared,ns_OptimSellar


# 3 Provide Discipline inputs

In [37]:
# Sellar inputs
local_dv = 10.
values_dict = {}
values_dict[f'{ns}.{sc_name}.x'] = 1.  
values_dict[f'{ns}.{sc_name}.y_1'] = 1.
values_dict[f'{ns}.{sc_name}.y_2'] = 1.
values_dict[f'{ns}.{sc_name}.z'] = array([1., 1.])
values_dict[f'{ns}.{sc_name}.Sellar_Problem.local_dv'] = local_dv
values_dict
my_dict = exec_eng.get_anonimated_data_dict()
pd.DataFrame.from_dict(values_dict,orient='index')

,0
doe.SellarDoeScenario.x,1.0
doe.SellarDoeScenario.y_1,1.0
doe.SellarDoeScenario.y_2,1.0
doe.SellarDoeScenario.z,"[1.0, 1.0]"
doe.SellarDoeScenario.Sellar_Problem.local_dv,10.0


In [38]:
#exec_eng.dm.set_values_from_dict(values_dict)
exec_eng.load_study_from_input_dict(values_dict)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


# 4 Run Doe scenario

In [39]:
res = exec_eng.execute()
# res is without interest (retourn not the expected values but probably default values ) !

INFO:SoS.EE:PROCESS EXECUTION doe STARTS...
INFO:gemseo.core.doe_scenario: 
INFO:gemseo.core.doe_scenario:*** Start DOE Scenario execution ***
INFO:gemseo.core.doe_scenario:SellarDoeScenario
   Disciplines: Sellar_Problem Sellar_2 Sellar_1
   MDOFormulation: MDF
   Algorithm: None
INFO:SoS.EE.SoSDOEScenario:{'doe_file': '.\\X_pd.csv', 'delimiter': ',', 'comments': '#', 'skiprows': 2, 'eval_jac': False, 'n_processes': 1, 'max_time': 0, 'wait_time_between_samples': 0.0}
INFO:gemseo.algos.doe.doe_lib:Optimization problem:
   Minimize: doe.SellarDoeScenario.obj(doe.SellarDoeScenario.x, doe.SellarDoeScenario.z)
   With respect to: doe.SellarDoeScenario.x, doe.SellarDoeScenario.z
INFO:gemseo.algos.driver_lib:DOE sampling:   0%|          | 0/100 [00:00<?, ?it]


my_options
{'doe_file': '.\\X_pd.csv', 'delimiter': ',', 'comments': '#', 'skiprows': 2, 'eval_jac': False, 'n_processes': 1, 'max_time': 0, 'wait_time_between_samples': 0.0}


INFO:gemseo.mda.mda:	1	1.000000e+00
INFO:gemseo.mda.mda:	2	1.955732e-01
INFO:gemseo.mda.mda:	3	3.745010e-02
INFO:gemseo.mda.mda:	4	2.219543e-03
INFO:gemseo.mda.mda:	5	4.204470e-06
INFO:gemseo.mda.mda:	6	1.331353e-11
INFO:gemseo.mda.mda:	1	9.375378e-01
INFO:gemseo.mda.mda:	2	1.848855e-01
INFO:gemseo.mda.mda:	3	3.592528e-02
INFO:gemseo.mda.mda:	4	2.281063e-03
INFO:gemseo.mda.mda:	5	3.021333e-06
INFO:gemseo.mda.mda:	6	9.930774e-12
INFO:gemseo.algos.driver_lib:DOE sampling:   2%|▏         | 2/100 [00:00<00:00, 638.57 it/sec, obj=110]
INFO:gemseo.mda.mda:	1	6.617839e-01
INFO:gemseo.mda.mda:	2	1.459518e-01
INFO:gemseo.mda.mda:	3	2.866878e-02
INFO:gemseo.mda.mda:	4	2.025604e-03
INFO:gemseo.mda.mda:	5	2.036568e-06
INFO:gemseo.mda.mda:	6	9.048144e-12
INFO:gemseo.mda.mda:	1	4.367449e-01
INFO:gemseo.mda.mda:	2	1.071167e-01
INFO:gemseo.mda.mda:	3	2.123486e-02
INFO:gemseo.mda.mda:	4	9.974994e-04
INFO:gemseo.mda.mda:	5	6.342035e-06
INFO:gemseo.mda.mda:	6	4.326168e-11
INFO:gemseo.algos.driver_lib:DOE

INFO:gemseo.mda.mda:	2	2.401811e-01
INFO:gemseo.mda.mda:	3	4.686967e-02
INFO:gemseo.mda.mda:	4	2.791101e-03
INFO:gemseo.mda.mda:	5	2.036745e-06
INFO:gemseo.mda.mda:	6	4.167220e-12
INFO:gemseo.mda.mda:	1	3.451960e+00
INFO:gemseo.mda.mda:	2	4.243300e-01
INFO:gemseo.mda.mda:	3	8.013566e-02
INFO:gemseo.mda.mda:	4	3.439378e-03
INFO:gemseo.mda.mda:	5	2.429816e-06
INFO:gemseo.mda.mda:	6	1.757197e-12
INFO:gemseo.algos.driver_lib:DOE sampling:  34%|███▍      | 34/100 [00:02<00:01, 37.48 it/sec, obj=76.5]
INFO:gemseo.mda.mda:	1	3.460779e-01
INFO:gemseo.mda.mda:	2	9.433252e-02
INFO:gemseo.mda.mda:	3	1.880801e-02
INFO:gemseo.mda.mda:	4	1.499387e-03
INFO:gemseo.mda.mda:	5	4.736531e-07
INFO:gemseo.mda.mda:	1	6.008166e-01
INFO:gemseo.mda.mda:	2	1.381801e-01
INFO:gemseo.mda.mda:	3	2.634698e-02
INFO:gemseo.mda.mda:	4	1.429545e-03
INFO:gemseo.mda.mda:	5	6.376662e-06
INFO:gemseo.mda.mda:	6	3.429246e-11
INFO:gemseo.algos.driver_lib:DOE sampling:  36%|███▌      | 36/100 [00:02<00:01, 35.31 it/sec, obj=35.7

INFO:gemseo.mda.mda:	1	1.294715e+00
INFO:gemseo.mda.mda:	2	2.267967e-01
INFO:gemseo.mda.mda:	3	4.404222e-02
INFO:gemseo.mda.mda:	4	2.634286e-03
INFO:gemseo.mda.mda:	5	2.599366e-06
INFO:gemseo.mda.mda:	6	6.048491e-12
INFO:gemseo.algos.driver_lib:DOE sampling:  66%|██████▌   | 66/100 [00:05<00:01, 19.92 it/sec, obj=88.7]
INFO:gemseo.mda.mda:	1	5.174560e+00
INFO:gemseo.mda.mda:	2	4.884947e-01
INFO:gemseo.mda.mda:	3	9.690643e-02
INFO:gemseo.mda.mda:	4	4.025889e-03
INFO:gemseo.mda.mda:	5	1.609725e-07
INFO:gemseo.mda.mda:	1	1.848685e+00
INFO:gemseo.mda.mda:	2	2.729755e-01
INFO:gemseo.mda.mda:	3	5.336177e-02
INFO:gemseo.mda.mda:	4	3.187705e-03
INFO:gemseo.mda.mda:	5	2.897213e-07
INFO:gemseo.algos.driver_lib:DOE sampling:  68%|██████▊   | 68/100 [00:05<00:01, 19.36 it/sec, obj=46.3]
INFO:gemseo.mda.mda:	1	1.578940e+00
INFO:gemseo.mda.mda:	2	2.500564e-01
INFO:gemseo.mda.mda:	3	4.921428e-02
INFO:gemseo.mda.mda:	4	2.976197e-03
INFO:gemseo.mda.mda:	5	1.086791e-06
INFO:gemseo.mda.mda:	6	1.952531e-1

INFO:gemseo.mda.mda:	3	9.960763e-02
INFO:gemseo.mda.mda:	4	3.814997e-03
INFO:gemseo.mda.mda:	5	1.566742e-06
INFO:gemseo.mda.mda:	6	6.658284e-13
INFO:gemseo.algos.driver_lib:DOE sampling:  98%|█████████▊| 98/100 [00:07<00:00, 12.83 it/sec, obj=113]
INFO:gemseo.mda.mda:	1	6.036291e-01
INFO:gemseo.mda.mda:	2	1.369595e-01
INFO:gemseo.mda.mda:	3	2.666302e-02
INFO:gemseo.mda.mda:	4	2.066306e-03
INFO:gemseo.mda.mda:	5	4.547889e-07
INFO:gemseo.mda.mda:	1	1.406017e+00
INFO:gemseo.mda.mda:	2	2.555851e-01
INFO:gemseo.mda.mda:	3	4.689056e-02
INFO:gemseo.mda.mda:	4	2.287045e-03
INFO:gemseo.mda.mda:	5	6.367944e-06
INFO:gemseo.mda.mda:	6	1.473268e-11
INFO:gemseo.algos.driver_lib:DOE sampling: 100%|██████████| 100/100 [00:07<00:00, 12.59 it/sec, obj=54.8]
INFO:gemseo.algos.driver_lib:Optimization result:
Objective value = 13.915272626028926
The result is feasible.
Status: None
Optimizer message: None
Number of calls to the objective function by the optimizer: 100
INFO:gemseo.algos.driver_lib:Design sp

Remark : all options that are None are no kept and so appear with a warning (this is the case of default None options thar are not setted with not None value)

# 5 Display Doe  Scenario results

In [40]:
# retrieve discipline to check the result...
doe_disc = exec_eng.dm.get_disciplines_with_name("doe." + sc_name)[0]

In [41]:
type(doe_disc)

sos_trades_core.execution_engine.sos_doe_scenario.SoSDOEScenario

In [42]:
doe_disc.get_disc_full_name()

'doe.SellarDoeScenario'

In [43]:
opt_P = doe_disc.opt_problem
print(opt_P)

Optimization problem:
   Minimize: doe.SellarDoeScenario.obj(doe.SellarDoeScenario.x, doe.SellarDoeScenario.z)
   With respect to: doe.SellarDoeScenario.x, doe.SellarDoeScenario.z


In [44]:
doe_disc_input = doe_disc.get_sosdisc_inputs()
pd.DataFrame.from_dict(doe_disc_input,orient='index')

,0
n_samples,100
algo,CustomDOE
design_space,variable value lower_bnd upper...
formulation,MDF
maximize_objective,False
objective_name,obj
differentiation_method,finite_differences
algo_options,"{'doe_file': '.\X_pd.csv', 'delimiter': ',', '..."
parallel_options,"{'parallel': False, 'n_processes': 8, 'use_thr..."
eval_mode,False


In [45]:
doe_disc_output = doe_disc.get_sosdisc_outputs()

In [46]:
# doe_disc_output

# 5.1 Table of computed points

In [47]:
XY_pd= doe_disc_output['doe_ds_io']

In [48]:
X_pd = XY_pd['design_parameters']
X_pd

doe.SellarDoeScenario.x doe.SellarDoeScenario.z          
                         0                       0         1
0                 9.379764                1.515949  5.631736
1                 8.886448               -1.196271  6.523436
2                 3.713714               -1.994758  4.822571
3                 0.041702                1.749067  3.617234
4                 6.954955               -9.316161  9.918161
..                     ...                     ...       ...
95                6.661678                5.719822  7.645988
96                2.358656               -6.389327  5.297002
97                3.011475                8.776000  9.837792
98                4.804455               -2.413481  1.131552
99                4.001988                2.954436  9.746571

[100 rows x 3 columns]

In [49]:
Y_pd = XY_pd['functions']
Y_pd

,doe.SellarDoeScenario.obj
,0
0,118.714541
1,110.493389
2,39.894420
3,18.808762
4,169.817929
...,...
95,105.062729
96,68.185137
97,113.215059


# 5.2 Optimum

In [50]:
my_optim_result = doe_disc_output['optim_result']
my_optim_result

{'x_opt': array([ 1.00983468, -0.65057567,  0.76923226]),
 'f_opt': 13.915272626028926}

In [51]:
my_optim_result['x_opt']

array([ 1.00983468, -0.65057567,  0.76923226])

In [52]:
my_optim_result['f_opt']

13.915272626028926

# 5.3 From gemseo 

In [53]:
print(opt_P.objective)
print(opt_P.constraints)
print(opt_P.differentiation_method)
print(opt_P.fd_step)

doe.SellarDoeScenario.obj(doe.SellarDoeScenario.x, doe.SellarDoeScenario.z)
[]
finite_differences
1e-06


In [54]:
dataset = opt_P.export_to_dataset("dataset_name")

In [55]:
XY_pd=dataset.export_to_dataframe()
XY_pd

design_parameters                                    \
   doe.SellarDoeScenario.x doe.SellarDoeScenario.z             
                         0                       0         1   
0                 9.379764                1.515949  5.631736   
1                 8.886448               -1.196271  6.523436   
2                 3.713714               -1.994758  4.822571   
3                 0.041702                1.749067  3.617234   
4                 6.954955               -9.316161  9.918161   
..                     ...                     ...       ...   
95                6.661678                5.719822  7.645988   
96                2.358656               -6.389327  5.297002   
97                3.011475                8.776000  9.837792   
98                4.804455               -2.413481  1.131552   
99                4.001988                2.954436  9.746571   

                   functions  
   doe.SellarDoeScenario.obj  
                           0  
0                 118.714541  
1                 110.493389  
2                  39.894420  
3                  18.808762  
4                 169.817929  
..                       ...  
95                105.062729  
96                 68.185137  
97                113.215059  
98                 45.682186  
99                 54.826284  

[100 rows x 4 columns]

In [56]:
X_pd = XY_pd['design_parameters']
X_pd

doe.SellarDoeScenario.x doe.SellarDoeScenario.z          
                         0                       0         1
0                 9.379764                1.515949  5.631736
1                 8.886448               -1.196271  6.523436
2                 3.713714               -1.994758  4.822571
3                 0.041702                1.749067  3.617234
4                 6.954955               -9.316161  9.918161
..                     ...                     ...       ...
95                6.661678                5.719822  7.645988
96                2.358656               -6.389327  5.297002
97                3.011475                8.776000  9.837792
98                4.804455               -2.413481  1.131552
99                4.001988                2.954436  9.746571

[100 rows x 3 columns]

In [57]:
Y_pd = XY_pd['functions']
Y_pd

,doe.SellarDoeScenario.obj
,0
0,118.714541
1,110.493389
2,39.894420
3,18.808762
4,169.817929
...,...
95,105.062729
96,68.185137
97,113.215059


In [58]:
import numpy as np
min(np.array(Y_pd['doe.SellarDoeScenario.obj']['0']))

13.915272626028926

In [59]:
doe_disc.optimization_result.f_opt

13.915272626028926

In [60]:
doe_disc.optimization_result.x_opt

array([ 1.00983468, -0.65057567,  0.76923226])

In [61]:
doe_disc.print_execution_metrics()

INFO:gemseo.core.scenario:* Scenario Executions statistics *
INFO:gemseo.core.scenario:* Discipline: Sellar_Problem
INFO:gemseo.core.scenario:Executions number: 100
INFO:gemseo.core.scenario:Execution time:  0.024357999999994107 s
INFO:gemseo.core.scenario:Linearizations number: 0
INFO:gemseo.core.scenario:* Discipline: Sellar_2
INFO:gemseo.core.scenario:Executions number: 558
INFO:gemseo.core.scenario:Execution time:  0.09646720000000109 s
INFO:gemseo.core.scenario:Linearizations number: 0
INFO:gemseo.core.scenario:* Discipline: Sellar_1
INFO:gemseo.core.scenario:Executions number: 558
INFO:gemseo.core.scenario:Execution time:  0.11474639999999425 s
INFO:gemseo.core.scenario:Linearizations number: 0
INFO:gemseo.core.scenario:Total number of executions calls 1216
INFO:gemseo.core.scenario:Total number of linearizations 0


In [62]:
doe_disc.get_optimum().__dict__

{'x_0': array([9.37976388, 1.51594904, 5.63173624]),
 'optimizer_name': 'CustomDOE',
 'x_opt': array([ 1.00983468, -0.65057567,  0.76923226]),
 'message': None,
 'f_opt': 13.915272626028926,
 'constraints_values': {},
 'constraints_grad': {},
 'status': None,
 'n_obj_call': 100,
 'n_grad_call': None,
 'n_constr_call': None,
 'is_feasible': True}

# 6 Set Doe optim scenario Custom DoE optim from sample

In [63]:
#my_formulation = "DisciplinaryOpt"
my_formulation = "MDF"
#my_formulation = "IDF"
#my_formulation =  "BiLevel "

In [64]:
my_doe_algo = "CustomDOE"

In [65]:
X_pd = pd.read_csv('.\\' + 'X_pd.csv')

In [66]:
X_pd

,doe.SellarDoeScenario.x,doe.SellarDoeScenario.z,doe.SellarDoeScenario.z.1
0,0.000000,0.000000,1.000000
1,9.379764,1.515949,5.631736
2,8.886448,-1.196271,6.523436
3,3.713714,-1.994758,4.822571
4,0.041702,1.749067,3.617234
...,...,...,...
96,6.661678,5.719822,7.645988
97,2.358656,-6.389327,5.297002
98,3.011475,8.776000,9.837792
99,4.804455,-2.413481,1.131552


In [67]:
import numpy as np
X_np = np.array(X_pd)
X_np

array([[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 9.37976388e+00,  1.51594904e+00,  5.63173624e+00],
       [ 8.88644794e+00, -1.19627054e+00,  6.52343621e+00],
       [ 3.71371357e+00, -1.99475780e+00,  4.82257093e+00],
       [ 4.17022005e-02,  1.74906689e+00,  3.61723405e+00],
       [ 6.95495479e+00, -9.31616110e+00,  9.91816129e+00],
       [ 3.99230245e+00, -5.79000931e+00,  6.69501761e+00],
       [ 4.68635419e+00,  6.17857774e+00,  5.93700842e+00],
       [ 9.65724885e+00, -8.16603392e+00,  7.80155333e+00],
       [ 8.58288458e+00, -9.53088785e+00,  5.41567914e+00],
       [ 8.03152448e+00, -5.21931962e+00,  8.46047161e+00],
       [ 8.39186018e+00,  3.38980326e+00,  1.20182883e+00],
       [ 5.92649196e+00, -5.88016462e-01,  2.06944002e+00],
       [ 1.15331653e+00, -4.81449828e+00,  2.96216957e+00],
       [ 1.30233257e-01, -5.91718880e+00,  2.29445948e+00],
       [ 7.03948756e+00,  5.11640284e+00,  6.30659611e+00],
       [ 9.03905478e-01,  8.39476042e+00

In [68]:
exec_eng = ExecutionEngine(study_name)
factory = exec_eng.factory
builder = factory.get_builder_from_process(repo=repo,mod_id=proc_name)
exec_eng.factory.set_builders_to_coupling_builder(builder)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


Possible algo options: 

In [69]:
from gemseo.algos.doe.doe_factory import DOEFactory
driver_lib = DOEFactory().create(my_doe_algo)
driver_lib.init_options_grammar(my_doe_algo)
algo_options_keys = list(driver_lib.opt_grammar.properties_dict.keys())
print(algo_options_keys)

['doe_file', 'samples', 'delimiter', 'comments', 'skiprows', 'eval_jac', 'n_processes', 'max_time', 'wait_time_between_samples']


In [70]:
#-- set up disciplines in Scenario
disc_dict = {}
# Doe inputs
disc_dict[f'{ns}.SellarDoeScenario.n_samples'] = 100
# 'lhs', 'CustomDOE', 'fullfact', ...
disc_dict[f'{ns}.SellarDoeScenario.algo'] = my_doe_algo
disc_dict[f'{ns}.SellarDoeScenario.design_space'] = dspace

disc_dict[f'{ns}.SellarDoeScenario.formulation'] = my_formulation
disc_dict[f'{ns}.SellarDoeScenario.objective_name'] = 'obj'
#disc_dict[f'{ns}.SellarDoeScenario.ineq_constraints'] = [f'c_1', f'c_2']
disc_dict[f'{ns}.SellarDoeScenario.algo_options'] = {
                                'samples': X_np
                                                    }
disc_dict

{'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x       [1.0]         [0.0]        [10.0]             True   
 1        z  [5.0, 2.0]  [-10.0, 0.0]  [10.0, 10.0]             True   
 2      y_1       [1.0]      [-100.0]       [100.0]             True   
 3      y_2       [1.0]      [-100.0]       [100.0]             True   
 
   activated_elem  
 0         [True]  
 1   [True, True]  
 2         [True]  
 3         [True]  ,
 'doe.SellarDoeScenario.formulation': 'MDF',
 'doe.SellarDoeScenario.objective_name': 'obj',
 'doe.SellarDoeScenario.algo_options': {'samples': array([[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
         [ 9.37976388e+00,  1.51594904e+00,  5.63173624e+00],
         [ 8.88644794e+00, -1.19627054e+00,  6.52343621e+00],
         [ 3.71371357e+00, -1.99475780e+00,  4.82257093e+00],
         [ 4.17022005e-0

In [71]:
my_dict = exec_eng.get_anonimated_data_dict()

In [72]:
#pd.DataFrame.from_dict(disc_dict,orient='index')
disc_dict

{'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x       [1.0]         [0.0]        [10.0]             True   
 1        z  [5.0, 2.0]  [-10.0, 0.0]  [10.0, 10.0]             True   
 2      y_1       [1.0]      [-100.0]       [100.0]             True   
 3      y_2       [1.0]      [-100.0]       [100.0]             True   
 
   activated_elem  
 0         [True]  
 1   [True, True]  
 2         [True]  
 3         [True]  ,
 'doe.SellarDoeScenario.formulation': 'MDF',
 'doe.SellarDoeScenario.objective_name': 'obj',
 'doe.SellarDoeScenario.algo_options': {'samples': array([[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
         [ 9.37976388e+00,  1.51594904e+00,  5.63173624e+00],
         [ 8.88644794e+00, -1.19627054e+00,  6.52343621e+00],
         [ 3.71371357e+00, -1.99475780e+00,  4.82257093e+00],
         [ 4.17022005e-0

In [73]:
#exec_eng.dm.set_values_from_dict(disc_dict)
exec_eng.load_study_from_input_dict(disc_dict)

INFO:SoS.EE:configuring ...
INFO:SoS.EE:configuring ...
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
ERROR:SoS.EE.DataManager:Variable: doe.SellarDoeScenario.x value is not set!
ERROR:SoS.EE.DataManager:Variable: doe.SellarDoeScenario.z value is not set!
ERROR:SoS.EE.DataManager:Variable: doe.SellarDoeScenario.Sellar_Problem.local_dv value is not set!


In [74]:
exec_eng.dm.get_data_dict_values()

{'doe.sub_mda_class': 'MDAJacobi',
 'doe.max_mda_iter': 30,
 'doe.n_processes': 1,
 'doe.chain_linearize': False,
 'doe.tolerance': 1e-06,
 'doe.use_lu_fact': False,
 'doe.warm_start': False,
 'doe.acceleration': 'm2d',
 'doe.warm_start_threshold': -1,
 'doe.n_subcouplings_parallel': 1,
 'doe.tolerance_gs': 10.0,
 'doe.relax_factor': 0.99,
 'doe.epsilon0': 1e-06,
 'doe.linear_solver_MDO': 'GMRES',
 'doe.linear_solver_MDO_preconditioner': 'None',
 'doe.linear_solver_MDO_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.linear_solver_MDA': 'GMRES',
 'doe.linear_solver_MDA_preconditioner': 'None',
 'doe.linear_solver_MDA_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.group_mda_disciplines': False,
 'doe.authorize_self_coupled_disciplines': False,
 'doe.linearization_mode': 'auto',
 'doe.cache_type': 'SimpleCache',
 'doe.cache_file_path': None,
 'doe.debug_mode': '',
 'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space

In [75]:
# Sellar inputs
local_dv = 10.
values_dict = {}
values_dict[f'{ns}.{sc_name}.x'] = 1.  
values_dict[f'{ns}.{sc_name}.y_1'] = 1.
values_dict[f'{ns}.{sc_name}.y_2'] = 1.
values_dict[f'{ns}.{sc_name}.z'] = array([1., 1.])
values_dict[f'{ns}.{sc_name}.Sellar_Problem.local_dv'] = local_dv
values_dict
my_dict = exec_eng.get_anonimated_data_dict()
pd.DataFrame.from_dict(values_dict,orient='index')


,0
doe.SellarDoeScenario.x,1.0
doe.SellarDoeScenario.y_1,1.0
doe.SellarDoeScenario.y_2,1.0
doe.SellarDoeScenario.z,"[1.0, 1.0]"
doe.SellarDoeScenario.Sellar_Problem.local_dv,10.0


In [76]:
#exec_eng.dm.set_values_from_dict(values_dict)
exec_eng.load_study_from_input_dict(values_dict)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


In [77]:
exec_eng.dm.get_data_dict_values()

{'doe.sub_mda_class': 'MDAJacobi',
 'doe.max_mda_iter': 30,
 'doe.n_processes': 1,
 'doe.chain_linearize': False,
 'doe.tolerance': 1e-06,
 'doe.use_lu_fact': False,
 'doe.warm_start': False,
 'doe.acceleration': 'm2d',
 'doe.warm_start_threshold': -1,
 'doe.n_subcouplings_parallel': 1,
 'doe.tolerance_gs': 10.0,
 'doe.relax_factor': 0.99,
 'doe.epsilon0': 1e-06,
 'doe.linear_solver_MDO': 'GMRES',
 'doe.linear_solver_MDO_preconditioner': 'None',
 'doe.linear_solver_MDO_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.linear_solver_MDA': 'GMRES',
 'doe.linear_solver_MDA_preconditioner': 'None',
 'doe.linear_solver_MDA_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.group_mda_disciplines': False,
 'doe.authorize_self_coupled_disciplines': False,
 'doe.linearization_mode': 'auto',
 'doe.cache_type': 'SimpleCache',
 'doe.cache_file_path': None,
 'doe.debug_mode': '',
 'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space

Problem : 'doe.SellarDoeScenario.algo_options' has 'doe_file' and not 'samples' has we would like 
It would work if we had algo_default_val = self.default_algo_options_CustomDOE[algo_option] in line 302 of sos_doe_senario.py


In [78]:
res = exec_eng.execute()
# res is without interest (retourn not the expected values but probably default values ) !

INFO:SoS.EE:PROCESS EXECUTION doe STARTS...
INFO:gemseo.core.doe_scenario: 
INFO:gemseo.core.doe_scenario:*** Start DOE Scenario execution ***
INFO:gemseo.core.doe_scenario:SellarDoeScenario
   Disciplines: Sellar_Problem Sellar_2 Sellar_1
   MDOFormulation: MDF
   Algorithm: None
INFO:SoS.EE.SoSDOEScenario:{'doe_file': 'X_pd.csv', 'delimiter': ',', 'comments': '#', 'skiprows': 0, 'eval_jac': False, 'n_processes': 1, 'max_time': 0, 'wait_time_between_samples': 0.0}
INFO:gemseo.algos.doe.doe_lib:Optimization problem:
   Minimize: doe.SellarDoeScenario.obj(doe.SellarDoeScenario.x, doe.SellarDoeScenario.z)
   With respect to: doe.SellarDoeScenario.x, doe.SellarDoeScenario.z
ERROR:gemseo.algos.doe.lib_custom:Failed to load DOE input file: X_pd.csv
ERROR:SoS.EE.SoSDOEScenario:could not convert string to float: 'doe.SellarDoeScenario.x'
Traceback (most recent call last):
  File "C:\Users\ng947ac\Documents\Daniel\git\workspace\sostrades-core\sos_trades_core\execution_engine\sos_discipline.py"

my_options
{'doe_file': 'X_pd.csv', 'delimiter': ',', 'comments': '#', 'skiprows': 0, 'eval_jac': False, 'n_processes': 1, 'max_time': 0, 'wait_time_between_samples': 0.0}


ValueError: could not convert string to float: 'doe.SellarDoeScenario.x'

In [ ]:
# retrieve discipline to check the result...
doe_disc = exec_eng.dm.get_disciplines_with_name("doe." + sc_name)[0]

In [ ]:
opt_P = doe_disc.opt_problem
print(opt_P)

In [ ]:
doe_disc_input = doe_disc.get_sosdisc_inputs()
pd.DataFrame.from_dict(doe_disc_input,orient='index')

In [ ]:
doe_disc_output = doe_disc.get_sosdisc_outputs()
XY_pd= doe_disc_output['doe_ds_io']

In [ ]:
my_optim_result = doe_disc_output['optim_result']
my_optim_result